## Bibliotecas

In [58]:
# Importações constantes

import pandas as pd
import re
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from optuna import create_study, Trial

In [59]:
# Coisas definidas previamente

# num de features
num_dados_de_entrada = 69

# num de targets
num_dados_de_saida = 1

# quantas vezes iremos rodar o optuna
NUM_TENTATIVAS = 110

# 10% dos dados gerais p/ separados para teste
TAMANHO_TESTE = 0.1

# 10% dos dados do treino p/ validar 
TAMANHO_VALIDACAO = 0.1

# Semente aleatória petista
SEMENTE_ALEATORIA = 13

NUM_EPOCAS = 60

## DataModule (completo)

In [60]:
class DataModule(L.LightningDataModule):
    def __init__(
        self,
        tamanho_lote=256,
        num_trabalhadores=2,
    ):
        super().__init__()

        self.tamanho_lote = tamanho_lote
        self.num_trabalhadores = num_trabalhadores

    def prepare_data(self):

        pd.read_csv("dataset_tratado.csv")

    def setup(self, stage):

        features = ["Thermodynamic stability level", "Energy", "Work function", "Heat of formation", "Space group number", "Volume of unit cell", "Electronegativity", "Be", "As", "O", "Ca", "Fe", "S", "In","Se","Sc","V","Zr","B","H","Te","Al","Mg","Ba","Pb","Mn","Si","Cr","Br","Ga","Hf","Ge","Ti","C","I","Li","Cl","Sr","Na","Nb","Ni","Ta","Pd","Pt","Tl","W","Sb","N","Cd","Cu","Sn","F","P","Ag","Au","Bi","Co","Zn","Rb","Os","Hg","Ir","Mo","Re","Rh","Ru","Y","Cs","K"]
        target = ["Band gap"]

        df = pd.read_csv("dataset_tratado.csv")
        
        df = df.reindex(features + target, axis=1)
        df = df.dropna()
        
        indices = df.index
        indices_treino_val, indices_teste = train_test_split(
            indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
        )

        df_treino_val = df.loc[indices_treino_val]
        df_teste = df.loc[indices_teste]

        indices = df_treino_val.index
        indices_treino, indices_val = train_test_split(
            indices,
            test_size=TAMANHO_TESTE,
            random_state=SEMENTE_ALEATORIA,
        )

        df_treino = df.loc[indices_treino]
        df_val = df.loc[indices_val]
        
        X_treino = df_treino.reindex(features, axis=1).values
        y_treino = df_treino.reindex(target, axis=1).values

        self.x_scaler = MaxAbsScaler()
        self.x_scaler.fit(X_treino)

        self.y_scaler = MaxAbsScaler()
        self.y_scaler.fit(y_treino)

        if stage == "fit":
            X_val = df_val.reindex(features, axis=1).values
            y_val = df_val.reindex(target, axis=1).values

            X_treino = self.x_scaler.transform(X_treino)
            y_treino = self.y_scaler.transform(y_treino)

            X_val = self.x_scaler.transform(X_val)
            y_val = self.y_scaler.transform(y_val)

            self.X_treino = torch.tensor(X_treino, dtype=torch.float32)
            self.y_treino = torch.tensor(y_treino, dtype=torch.float32)

            self.X_val = torch.tensor(X_val, dtype=torch.float32)
            self.y_val = torch.tensor(y_val, dtype=torch.float32)

        if stage == "test":
            X_teste = df_teste.reindex(features, axis=1).values
            y_teste = df_teste.reindex(target, axis=1).values

            X_teste = self.x_scaler.transform(X_teste)
            y_teste = self.y_scaler.transform(y_teste)

            self.X_teste = torch.tensor(X_teste, dtype=torch.float32)
            self.y_teste = torch.tensor(y_teste, dtype=torch.float32)

    def train_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_treino, self.y_treino),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

    def val_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_val, self.y_val),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

    def test_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_teste, self.y_teste),
            batch_size=self.tamanho_lote,
            num_workers=self.num_trabalhadores,
        )

## Criando a rede neural com PyTorch Lightning

In [61]:
class MLP(L.LightningModule):
    def __init__(
        self, num_camadas, num_neuronios, funcao_de_ativacao, otimizador, taxa_de_aprendizado, num_dados_entrada, num_targets
    ):
    
    # def __init__(
    #     self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    # ):
        super().__init__()
        
        camadas = []
        
        camadas.append(nn.Linear(num_dados_entrada, num_neuronios))
        camadas.append(funcao_de_ativacao)
        
        for _ in range(num_camadas-1):
            camadas.append(nn.Linear(num_neuronios, num_neuronios))
            camadas.append(funcao_de_ativacao)
            
        camadas.append(nn.Linear(num_neuronios, num_targets))
        
        
        self.todas_camadas = nn.Sequential(*camadas)
        
        self.otimizador = otimizador
        self.taxa_de_aprendizado = taxa_de_aprendizado

        # self.camadas = nn.Sequential(
        #     nn.Linear(num_dados_entrada, neuronios_c1),
        #     nn.Sigmoid(),
        #     nn.Linear(neuronios_c1, neuronios_c2),
        #     nn.Sigmoid(),
        #     nn.Linear(neuronios_c2, num_targets),
        # )

        self.fun_perda = F.mse_loss

        self.perdas_treino = []
        self.perdas_val = []

        self.curva_aprendizado_treino = []
        self.curva_aprendizado_val = []

    def forward(self, x):
        x = self.todas_camadas(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("loss", loss, prog_bar=True)
        self.perdas_treino.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("val_loss", loss, prog_bar=True)
        self.perdas_val.append(loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("test_loss", loss)

        return loss

    def on_train_epoch_end(self):
        # Atualiza curva de aprendizado
        perda_media = torch.stack(self.perdas_treino).mean()
        self.curva_aprendizado_treino.append(float(perda_media))
        self.perdas_treino.clear()

    def on_validation_epoch_end(self):
        # Atualiza curva de aprendizado
        perda_media = torch.stack(self.perdas_val).mean()
        self.curva_aprendizado_val.append(float(perda_media))
        self.perdas_val.clear()

    def configure_optimizers(self):
        if self.otimizador == "SGD":
            optimizer = optim.SGD(self.parameters(), lr=self.taxa_de_aprendizado)
        elif self.otimizador == "ADAM":
            optimizer = optim.Adam(self.parameters(), lr=self.taxa_de_aprendizado)
        elif self.otimizador == "RMSPROP":
            optimizer = optim.RMSprop(self.parameters(), lr=self.taxa_de_aprendizado)
        # else:
#             fazer erro
            
        return optimizer

## Treinando a rede (agora vamos engavetar p/ fazer com optuna)

In [62]:
# criando um treinador

# treinador = L.Trainer(max_epochs=NUM_EPOCAS)

In [63]:
# Criando uma instância da classe DataModule.

# dm = DataModule()

In [64]:
# Criando uma instância da rede neural.

# num_dados_de_entrada = 69
# num_dados_de_saida = 1
# neuronios_c1 = 3
# neuronios_c2 = 2

# minha_mlp = MLP(
#     num_dados_de_entrada, neuronios_c1, neuronios_c2, num_dados_de_saida
# )

In [65]:
# Podemos rodar o método fit do nosso treinador na nossa instância da rede neural, fornecendo também uma instância do DataModule como argumento.
# t0 = time.time()

# treinador.fit(minha_mlp, dm)

# tf = time.time()
# print(f'{tf - t0} segundos')

## Curva de aprendizado(engavetado tbm)

In [66]:
# figura, eixo = plt.subplots()

# ca_treino = minha_mlp.curva_aprendizado_treino
# ca_val = minha_mlp.curva_aprendizado_val

# eixo.plot(ca_treino, label="Treino")
# eixo.plot(ca_val, label="Validação")

# eixo.legend()

# eixo.set_xlim(left=0)

# eixo.set_title("Curva de aprendizado")
# eixo.set_xlabel("Época")
# eixo.set_ylabel("Loss");

## Testando a rede(engavetado tbm)

Antes de testar a rede neural que treinamos precisamos deixá-la no modo de avaliação.

In [67]:
# minha_mlp.eval() 

Antes de computar a métrica, temos que configurar a instância do DataModule no modo teste, criar um ambiente onde não computamos os gradientes locais (ambiente no_grad), computar os valores previstos do target e, finalmente, desnormalizar os dados com o inverse_transform.

In [68]:
# dm.setup("test")

# with torch.no_grad():
#     X_true = dm.X_teste

#     y_true = dm.y_teste
#     y_true = dm.y_scaler.inverse_transform(y_true)

#     y_pred = minha_mlp(X_true)
#     y_pred = dm.y_scaler.inverse_transform(y_pred)

#     RMSE = mean_squared_error(y_true, y_pred, squared=False)

#     print(RMSE)

In [69]:
# def bandgap_prediction(formula, parameters):
    
#     elements = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu',
#              'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Cs', 'Ba', 'Hf',
#              'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi']
    
#     part = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
#     material = dict((el, int(num) if num else 1) for el, num in part)
    
#     element_list = []
    
#     if set(material.keys()).issubset(set(elements)):
    
#         for element in elements:
#             if element in material:
#                 element_list.append(material[element])
#             else:
#                 element_list.append(0)
#     else:
#         print("Não é possível utilizar esse material, pois nem todos os elementos estão presentes no banco de dados.")

#     features = parameters + element_list
    
#     features_tensor = torch.tensor([features], dtype=torch.float32)

#     model = minha_mlp
    
#     model.eval()

#     with torch.no_grad():
#         exit = model(features_tensor)

#     prediction = exit.item()

#     print("Previsão do Band Gap:", prediction)

In [70]:
# formula = "Zr2P2Se6"
# parameters = [3.0, -53.151, 4.916, -0.672, 2.0, 755.942, 2.234]

# bandgap_prediction(formula, parameters)

## Salvando o modelo(engavetado tbm)

Salvando apenas os pesos e vieses da rede neural em um arquivo no seu computador. Para isso, podemos utilizar o módulo pickle.

In [71]:
# nome_arquivo = "meu_modelo.p"
# pickle.dump(minha_mlp.state_dict(), open(nome_arquivo, "wb"))

## Otimização de hipercondríacos  (Optuna)

In [72]:
def cria_instancia_modelo(trial):
    
    todas_funcoes_ativacao = {
        "Sigmoide" : nn.Sigmoid(),
        "Relu" : nn.ReLU(),
        "Tangente_Hiperbólica" : nn.Tanh(),
        "ELU" : nn.ELU(),
        "LeakyReLU" : nn.LeakyReLU()
    }

    num_neuronios = trial.suggest_int("num_neuronios", 30, 1000)  
    num_camadas = trial.suggest_int("num_camadas", 3, 10)
    funcao_de_ativacao = trial.suggest_categorical("funcao_de_ativacao", ["Sigmoide", "Relu", "Tangente_Hiperbólica", "ELU", "LeakyReLU"])

    taxa_de_aprendizado = trial.suggest_float("taxa_de_aprendizado", 0.000001, 0.5, log=True)
    
    otimizador = trial.suggest_categorical("otimizador", ["SGD", "ADAM", "RMSPROP"])
    
    # pegando a função paitonica da função de ativação dada pelo sugest de "funcao_de_ativacao"
    real_funcao_ativacao = todas_funcoes_ativacao[funcao_de_ativacao]
            

    model = MLP(num_camadas = num_camadas, num_neuronios = num_neuronios, funcao_de_ativacao = real_funcao_ativacao, otimizador = otimizador, taxa_de_aprendizado = taxa_de_aprendizado, num_dados_entrada = num_dados_de_entrada, num_targets = num_dados_de_saida)

    return model



In [73]:
# Criando a função objetivo

def funcao_objetivo(
    trial
):
    modelo = cria_instancia_modelo(trial)
    
#     P/ rodar no computador normal:
    # treinador = L.Trainer(max_epochs=NUM_EPOCAS)
    
#     P/ GPU:
    treinador = L.Trainer(max_epochs=NUM_EPOCAS, accelerator="gpu")
    
    dm = DataModule()
    
    treinador.fit(modelo, dm)
    
    modelo.eval()
    dm.setup("test")

    with torch.no_grad():
        X_true = dm.X_teste

        y_true = dm.y_teste
        y_true = dm.y_scaler.inverse_transform(y_true)

        y_pred = modelo(X_true)
        y_pred = dm.y_scaler.inverse_transform(y_pred)

        RMSE = mean_squared_error(y_true, y_pred, squared=False)

    return RMSE

In [74]:
# criando obj de estudo

objeto_de_estudo = create_study(direction="minimize")

[I 2024-05-11 20:50:56,216] A new study created in memory with name: no-name-5bcd4cd3-2096-490e-8889-9ba958ffeb45


In [75]:
objeto_de_estudo.optimize(funcao_objetivo, n_trials=NUM_TENTATIVAS)

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=20` reached.
[I 2024-05-12 02:04:41,537] Trial 109 finished with value: 0.8295059556787803 and parameters: {'num_neuronios': 769, 'num_camadas': 3, 'funcao_de_ativacao': 'Relu', 'taxa_de_aprendizado': 0.00011605249611663851, 'otimizador': 'ADAM'}. Best is trial 100 with value: 0.8109135873250647.


In [76]:
# observando os resultados

df = objeto_de_estudo.trials_dataframe()

df

,number,value,datetime_start,datetime_complete,duration,params_funcao_de_ativacao,params_num_camadas,params_num_neuronios,params_otimizador,params_taxa_de_aprendizado,state
0,0,8.971960e-01,2024-05-11 20:50:56.256863,2024-05-11 20:53:42.586276,0 days 00:02:46.329413,Tangente_Hiper,6,823,RMSPROP,0.000005,COMPLETE
1,1,1.029134e+16,2024-05-11 20:53:42.598370,2024-05-11 20:57:21.083899,0 days 00:03:38.485529,Eluércio,6,983,RMSPROP,0.091475,COMPLETE
2,2,1.035441e+00,2024-05-11 20:57:21.083899,2024-05-11 21:00:21.924433,0 days 00:03:00.840534,Sigmoide,9,812,SGD,0.002385,COMPLETE
3,3,1.058701e+00,2024-05-11 21:00:21.937955,2024-05-11 21:02:58.540363,0 days 00:02:36.602408,ElerisonGleysson,9,319,ADAM,0.000002,COMPLETE
4,4,8.790859e-01,2024-05-11 21:02:58.552828,2024-05-11 21:06:26.259232,0 days 00:03:27.706404,Eluércio,9,631,ADAM,0.000861,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...
105,105,8.440832e-01,2024-05-12 01:50:38.325018,2024-05-12 01:53:28.123461,0 days 00:02:49.798443,Relu,3,847,ADAM,0.000258,COMPLETE
106,106,8.933574e-01,2024-05-12 01:53:28.132011,2024-05-12 01:56:21.548160,0 days 00:02:53.416149,Relu,4,792,ADAM,0.000400,COMPLETE
107,107,8.137059e-01,2024-05-12 01:56:21.560164,2024-05-12 01:59:12.153175,0 days 00:02:50.593011,Relu,3,874,ADAM,0.000119,COMPLETE
108,108,9.039176e-01,2024-05-12 01:59:12.162671,2024-05-12 02:01:58.087939,0 days 00:02:45.925268,Tangente_Hiper,3,834,ADAM,0.000981,COMPLETE


In [77]:
# melhor trial achando pelo optuna, ou seja, os melhores hiperparametros achados para a sua rede

melhor_trial = objeto_de_estudo.best_trial

In [78]:
melhor_trial

FrozenTrial(number=100, values=[0.8109135873250647], datetime_start=datetime.datetime(2024, 5, 12, 1, 37, 13, 265341), datetime_complete=datetime.datetime(2024, 5, 12, 1, 39, 54, 354747), params={'num_neuronios': 719, 'num_camadas': 3, 'funcao_de_ativacao': 'Relu', 'taxa_de_aprendizado': 0.00015920855476766, 'otimizador': 'ADAM'}, distributions={'num_neuronios': IntDistribution(high=1000, log=False, low=30, step=1), 'num_camadas': IntDistribution(high=10, log=False, low=3, step=1), 'funcao_de_ativacao': CategoricalDistribution(choices=('Sigmoide', 'Relu', 'Tangente_Hiper', 'Eluércio', 'ElerisonGleysson')), 'taxa_de_aprendizado': FloatDistribution(high=0.5, log=True, low=1e-06, step=None), 'otimizador': CategoricalDistribution(choices=('SGD', 'ADAM', 'RMSPROP'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=100, state=TrialState.COMPLETE, value=None)

# Rodado na GPU: o melhor trial foi:
{'num_neuronios': 515, 'num_camadas': 3, 'funcao_de_ativacao': 'Relu', 'taxa_de_aprendizado': 8.000226694893512e-05, 'otimizador': 'ADAM'}

In [80]:
melhor_model_gpu = MLP(num_camadas = 3, num_neuronios = 515, funcao_de_ativacao = nn.ReLU(), otimizador = 'ADAM', taxa_de_aprendizado = 8.000226694893512e-05, num_dados_entrada = num_dados_de_entrada, num_targets = num_dados_de_saida)

Agora, vamos treinar nossa rede com nosso melhor trial. E vamos fazer a análise de qual a melhor quantidade de épocas, para isso vamos treinar a rede com 100 épocas e plocar a curva de aprendizado e assim escolher um número de épocas razoável.

In [81]:
# criando um treinador

treinador = L.Trainer(max_epochs=100)

# Criando uma instância da classe DataModule.

dm = DataModule()

# Podemos rodar o método fit do nosso treinador na nossa instância da rede neural, fornecendo também uma instância do DataModule como argumento.
t0 = time.time()

treinador.fit(melhor_model_gpu, dm)

tf = time.time()
print(f'{tf - t0} segundos')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type       | Params
---------------------------------------------
0 | todas_camadas | Sequential | 568 K 
---------------------------------------------
568 K     Trainable params
0         Non-trainable params
568 K     Total params
2.272     Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

c:\venv\ilumpy\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\venv\ilumpy\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
c:\venv\ilumpy\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=100` reached.


915.7958052158356 segundos


In [82]:
melhor_model_gpu.eval()
dm.setup("test")

with torch.no_grad():
    X_true = dm.X_teste

    y_true = dm.y_teste
    y_true = dm.y_scaler.inverse_transform(y_true)

    y_pred = melhor_model_gpu(X_true)
    y_pred = dm.y_scaler.inverse_transform(y_pred)

    RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(RMSE)

0.8527140257660798


In [83]:
# plotando a curva de apren.

figura, eixo = plt.subplots()

ca_treino = melhor_model_gpu.curva_aprendizado_treino
ca_val = melhor_model_gpu.curva_aprendizado_val

eixo.plot(ca_treino, label="Treino")
eixo.plot(ca_val, label="Validação")

eixo.legend()

eixo.set_xlim(left=0)

eixo.set_title("Curva de aprendizado")
eixo.set_xlabel("Época")
eixo.set_ylabel("Loss");

Agora sim! Podemos achar um números de épocas racionalmete razoável.


# Padrão ouro
Agora vamos treinar nossa rede com os melhores cenários que encontramos.

In [90]:
model_padrao_ouro = MLP(num_camadas = 3, num_neuronios = 515, funcao_de_ativacao = nn.ReLU(), otimizador = 'ADAM', taxa_de_aprendizado = 8.000226694893512e-05, num_dados_entrada = num_dados_de_entrada, num_targets = num_dados_de_saida)

In [91]:
# criando um treinador
treinador = L.Trainer(max_epochs=55)

# Criando uma instância da classe DataModule.
dm = DataModule()

# Podemos rodar o método fit do nosso treinador na nossa instância da rede neural, fornecendo também uma instância do DataModule como argumento.
t0 = time.time()

treinador.fit(model_padrao_ouro, dm)

tf = time.time()
print(f'{tf - t0} segundos')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type       | Params
---------------------------------------------
0 | todas_camadas | Sequential | 568 K 
---------------------------------------------
568 K     Trainable params
0         Non-trainable params
568 K     Total params
2.272     Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

c:\venv\ilumpy\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\venv\ilumpy\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
c:\venv\ilumpy\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=55` reached.


475.87007212638855 segundos


Vendo o quão bom é nosso modelo com o RMSE:

In [92]:
model_padrao_ouro.eval()
dm.setup("test")

with torch.no_grad():
    X_true = dm.X_teste

    y_true = dm.y_teste
    y_true = dm.y_scaler.inverse_transform(y_true)

    y_pred = model_padrao_ouro(X_true)
    y_pred = dm.y_scaler.inverse_transform(y_pred)

    RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(RMSE)

0.8679719322607771


Salvando apenas os pesos e vieses da rede neural em um arquivo no seu computador. Para isso, podemos utilizar o módulo pickle.

In [93]:
nome_arquivo = "modelo_final_BG.p"
pickle.dump(model_padrao_ouro.state_dict(), open(nome_arquivo, "wb"))

# Referências

1. CASSAR, D. R. PyTorch Lightning. (2024)
2. CASSAR, D. R.Treinando uma rede neural com pytorch. (2024)
3. CASSAR, D. R. Redes neurais artificiais do zero em Python. (2024)
4. CASSAR, D. R. Avaliação de modelos: a estratégia de divisão entre treino e teste. (2023)
5. CASSAR, D. R. Transformação e normalização. (2023)
6. CASSAR, D. R. Conversão simbólico-numérico. (2023)
7. ChatGPT para ajuda na resolução de bugs.